In [1]:
# REQUIREMENTS
#   - see https://www.mosek.com/products/academic-licenses/ to get a .lic file and place it in the required directory
#   - then "pip install mosek"

import os
if os.path.basename(os.getcwd()) != 'Learning-Dynamic-Systems': os.chdir(".\\..") 

import numpy as np
from optimal_controller.state_space import generate_perturbed
from optimal_controller.optimal_controls import *
import logging
logging.basicConfig(level=logging.INFO, force=True)

In [2]:
def generate_perturbed(matrix, n_samples, pct):
    """
    Generates a list of perturbed versions of a given matrix (A or B).
    The original matrix is always included, stored as the first element of the list.
    Perturbations are applied assuming OCF state space matrices, i.e. if an A matrix, only the final column is perturbed.

    Parameters:
    ----------
    matrix : np.ndarray
        The original matrix to perturb. Should be either square (interpreted as an A matrix) or Nx1 rectangular (interpreted as a B matrix).
    n_samples : int
        Number of perturbed matrices to generate, including the original matrix.
    pct : float
        Maximum percentage perturbation (relative to the absolute value of each element) applied to the matrix.
        For example, pct = 0.2 allows ±20% perturbations.

    Returns:
    -------
    perturbed_matrices : list of np.ndarray
        List containing the original matrix and (n_samples - 1) perturbed versions. Perturbations are random but bounded as follows:
        - For A matrices (square), only the last column is perturbed.
        - For B matrices (non-square), all elements are perturbed independently.
        This corresponds to perturbing the transfer function coefficients.
    """    
    display_generated_matrices = False

    if matrix.shape[0] == matrix.shape[1]:
        # Square matrix (A matrix)
        n_states = matrix.shape[0]
        delta_A_min = -pct * np.hstack((np.zeros((n_states, n_states - 1)), np.abs(matrix[:, n_states - 1:n_states])))
        delta_A_max = pct * np.hstack((np.zeros((n_states, n_states - 1)), np.abs(matrix[:, n_states - 1:n_states])))
        perturbed_matrices = [matrix]
        for _ in range(1, n_samples):
            delta_A = delta_A_min + (delta_A_max - delta_A_min) * np.random.randn(n_states, n_states)
            perturbed_matrices.append(matrix + delta_A)
    else:
        # Non-square matrix (B matrix)
        n_states, n_inputs = matrix.shape
        delta_B_min = -pct * np.abs(matrix)
        delta_B_max = pct * np.abs(matrix)
        perturbed_matrices = [matrix]
        for _ in range(1, n_samples):
            delta_B = delta_B_min + (delta_B_max - delta_B_min) * np.random.randn(n_states, n_inputs)
            perturbed_matrices.append(matrix + delta_B)
    
    if display_generated_matrices:
        for i, mat in enumerate(perturbed_matrices):
            print(f'perturbed_matrices[{i}] =\n', mat)
    
    return np.array(perturbed_matrices)


In [3]:
# n_states = 1
# n_inputs = 1
# A0 = np.array([[0.7]])
# B0 = np.array([[0.4]])
# n_plants = 2000
# A_list = generate_perturbed(A0, n_plants, 0.1)
# B_list = np.ones_like(A_list) * 0.4

In [4]:
n_states = 2
n_inputs = 2
A0 = np.array([[1, 0.5],
               [0.5, 1]])
B0 = np.eye(2)
n_plants = 128
A_list = generate_perturbed(A0, n_plants, 0.1)
B_list = generate_perturbed(B0, n_plants, 0.1)

In [5]:
# n_states = 4
# n_inputs = 1
# A0 = np.array([[0, 0, 0, -0.048],
#                [1, 0, 0, 0.676],
#                [0, 1, 0, -2.21],
#                [0, 0, 1, 2.6]])
# B0 = np.array([[-0.12], [0.68], [-1.4], [1]])
# n_plants = 128
# A_list = generate_perturbed(A0, n_plants, 0.005)
# B_list = generate_perturbed(B0, n_plants, 0.005)

In [6]:
# n_states = 6
# n_inputs = 1
# A0 = np.array([[0,  0,  0,  0,  0, -0.085],
#                [1,  0,  0,  0,  0,  0.642],
#                [0,  1,  0,  0,  0, -2.219],
#                [0,  0,  1,  0,  0,  4.354],
#                [0,  0,  0,  1,  0, -5.11 ],
#                [0,  0,  0,  0,  1,  3.4  ]])
# B0 = np.array([[-0.0068], [0.0796], [-0.4170], [1.17], [-1.7], [1]])
# n_plants = 256
# A_list = generate_perturbed(A0, n_plants, 0.005)
# B_list = generate_perturbed(B0, n_plants, 0.002)

In [7]:
K = get_optimal_controller(A_list, B_list)

INFO:optimal_controller.optimal_controls: Defined 23 LMI constraints


INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.987 0.464]
 [0.506 0.970]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.687 0.594]
 [0.467 0.925]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 7.6474
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 6.7194


In [8]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 128 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.998 0.463]
 [0.534 0.964]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.833 0.628]
 [0.381 0.918]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 7.5838
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 6.7234


In [9]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_oliveira'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Convex hull of A set has 11 vertices
INFO:optimal_controller.optimal_controls: Convex hull of B set has 2 vertices
INFO:optimal_controller.optimal_controls: This will lead to 22 LMI constraints
INFO:optimal_controller.optimal_controls: Defined 22 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[1.010 0.514]
 [0.461 0.935]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[1.010 0.514]
 [0.461 0.935]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 7.0197
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 7.0197


In [10]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_peaucelle'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'vertex_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 23 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.987 0.464]
 [0.506 0.970]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.687 0.594]
 [0.467 0.925]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 7.6474
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 6.7194


In [11]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'riccati'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Riccati heuristic failed to find a simultaneously stabilising feedback gain!


AssertionError: 

In [ ]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'lowresMVEE'
ocSolver.set_max_n_verts(1000)
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 114 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[1.007 0.449]
 [0.497 0.976]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.847 0.707]
 [0.512 0.867]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.3124
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8529
